# Intelligence artificielle - Automne 2025 - Laboratoire 10

## First Order Predicate Logic : Réseaux sémantiques


## Introduction

L'objectif de ce laboratoire est de comprendre les concepts liés aux réseaux sémantiques et de mettre en œuvre des mécanismes simples pour déterminer les attributs des objets dans un réseau sémantique.

### Classes, attributs et objets

Nous considérons que tout **objet** peut appartenir à une **classe**. Cette association est décrite par la relation $ Object \xrightarrow{isa} Class $. Nous représenterons cette relation par une instance du prédicat $ isa $, sous la forme $ isa (Object, Class) $.

Une classe peut **hériter** une autre classe (c'est un sous-type). Cette relation est appelée A-Kind-Of et est décrit comme $ Subclass \xrightarrow{ako} Class $. Nous représenterons cette relation par une instance du prédicat $ ako $, sous la forme $ ako (Subclass, Class) $.

Une classe peut avoir des **attributs** qui ont des **valeurs**. Nous représentons qu'une classe $ Class $ a l'attribut $ Atr $ avec la valeur $ val $ en tant qu'atome $ Atr (Class, value) $. On va proceder de la même manière pour un objet.

Les attributs sont hérités d'une classe à une sous-classe de celle-ci, et de la classe à l'objet (instance). Pour déterminer la valeur d'un attribut pour un objet particulier, il faut remonter (vers les superclasses) sur $ isa $ et $ ako $ pour trouver la première définition (la plus proche) de la valeur de cet attribut, dans un ancêtre de la classe d'objets.

Nous représenterons un réseau sémantique comme une base de connaissances contenant exclusivement des atomes. Les prédicats seront $ isa $, $ ako $, et les prédicats correspondant aux différents attributs.

## Fonctions utiles des laboratoires précédents

### Tâche 0

Sauvez la solution de `lab 8` (*Representation et Unification*) avec le nom `Lab08.py`. Nous utiliserons également les fonctions déjà mises en œuvre dans `Devoir3_Resolution`.

In [5]:
from Lab08 import make_var, make_const, make_atom, make_or, make_neg, \
                is_variable, is_constant, is_atom, is_function_call, \
                get_args, get_head, get_name, get_value, print_formula
from LPTester import testA, testFKB, testL, test_batch, testBatch

dummy = make_atom("P")
[and_name, or_name, neg_name] = [get_head(s) for s in [make_and(dummy, dummy), make_or(dummy, dummy), make_neg(dummy)]]
def pFail(message, f):
    print(message + " <" + str(f) + ">")
    return False
def check_term(T):
    if is_constant(T):
        return (get_value(T) is not None) or pFail("The value of the constant is None", T)
    if is_variable(T):
        return (get_name(T) is not None) or pFail("The name of the variable is None", T)
    if is_function_call(T):
        return not [t for t in get_args(T) if not check_term(t)] and \
            (get_head(T) is not None or pFail("Function is not callable", T))
    return pFail("Term is not one of constant, variable or function call", T)
def check_atom(A):
    if is_atom(A):
        return not [t for t in get_args(A) if not check_term(t)] and \
            (get_head(A) is not None or pFail("Predicate name is None", A))
    return pFail("Is not an atom", A)
def check_sentence(S):
    if is_atom(S):
        return check_atom(S)
    if is_sentence(S):
        if get_head(S) in [and_name, or_name]:
            return (len(get_args(S)) >= 2 or pFail("Sentence has too few operands", S)) \
                and not [s for s in get_args(S) if not check_sentence(s)]
        if get_head(S) == neg_name:
            return (len(get_args(S)) == 1 or pFail("Negative sentence has not just 1 operand", S)) \
                and check_sentence(get_args(S)[0])
    return pFail("Not sentence or unknown type", S)

def add_statement(kb, conclusion, *hypotheses):
    s = conclusion if not hypotheses else make_or(*([make_neg(s) for s in hypotheses] + [conclusion]))
    if check_sentence(s):
        kb.append(s)
        print("OK: Added statement " + print_formula(s, True))
        return True
    print("-- FAILED CHECK: Sentence does not check out <"+print_formula(s, True)+"><" + str(s) + ">")
    return False
def print_KB(KB):
    print("KB now:")
    for s in KB:
        print("\t\t\t" + print_formula(s, True))

## Base de connaissances

Nous avons les affirmations suivantes:

1. Most cars, but not all, are powered by an internal combustion engine (ICE).
2. An ICE provides a typical car with about 110 HP.
3. A hybrid car is a car that has an ICE and an electric motor.
4. There are two categorizations for hybrid cars:
5. you can have parallel versus series hybrids;
6. and you can have mild versus full hybrids.
7. A hybrid would typically provide 100HP from the ICE and 30HP from the electric motor.
8. Some cars can be assisted by a low power electric motor (about 20HP).
9. A mild hybrid is a hybrid that has an ICE assisted by such a low power electric motor.
10. An electric car is considered to be able to be powered by electric power alone.
11. Electric cars have about 120HP, and no ICE.
12. A full hybrid can be powered only by its electric motor, so it could be considered a kind of electric car.
13. Full hybrids feature electric motors of about 100HP.
14. The Toyota Prius is a full and parallel hybrid.
15. The Honda Insight is a mild parallel hybrid.
16. The Chevrolet Volt is a full, series hybrid.
17. The Nissan Leaf is an electric car.

### Tâche 1

Dans la description de la base de connaissances ci-dessous, ajoutez les phrases logiques correspondant aux énoncés indiqués.

In [2]:
hybrids_KB = []
# 1. Most cars, but not all, are powered by an internal combustion engine (ICE).
# 2. An ICE provides a typical car with about 110 HP.
add_statement(hybrids_KB, make_atom("gas-power", make_const("car"), make_const("110")))
# 3. A hybrid car is a car that has an ICE and an electric motor.
add_statement(hybrids_KB, make_atom("ako", make_const("hybrid-car"), make_const("car")))
# 4. There are two categorizations for hybrid cars:
# 5. you can have parallel versus series hybrids (part 1)
add_statement(hybrids_KB, make_atom("ako", make_const("parallel-hybrid"), make_const("hybrid-car")))
# 5. you can have parallel versus series hybrids (part 2)
add_statement(hybrids_KB, make_atom("ako", make_const("series-hybrid"), make_const("hybrid-car")))
# 6. and you can have mild versus full hybrids. (part 1)
add_statement(hybrids_KB, make_atom("ako", make_const("mild-hybrid"), make_const("hybrid-car")))
# 6. and you can have mild versus full hybrids. (part 2)
add_statement(hybrids_KB, make_atom("ako", make_const("full-hybrid"), make_const("hybrid-car")))
# 7. A hybrid would typically provide 100HP from the ICE and 30HP from the electric motor. (part 1)
add_statement(hybrids_KB, make_atom("electric-power", make_const("hybrid-car"), make_const("30")))
# 7. A hybrid would typically provide 100HP from the ICE and 30HP from the electric motor. (part 2)
add_statement(hybrids_KB, make_atom("gas-power", make_const("hybrid-car"), make_const("100")))
# 8. Some cars can be assisted by a low power electric motor (about 20HP) (part 1)
add_statement(hybrids_KB, make_atom("ako", make_const("assisted-electric"), make_const("car")))
# 8. Some cars can be assisted by a low power electric motor (about 20HP) (part 2)
add_statement(hybrids_KB, make_atom("electric-power", make_const("assisted-electric"), make_const("20")))
# 9. A mild hybrid is a hybrid that has an ICE assisted by such a low power electric motor.
add_statement(hybrids_KB, make_atom("ako", make_const("mild-hybrid"), make_const("assisted-electric")))
# 10. An electric car is a car powered by electric power alone.
# TODO
# 11. Electric cars have about 120HP, and no ICE. (part 1)
# TODO
# 11. Electric cars have about 120HP, and no ICE. (part 2)
# TODO
# 12. A full hybrid can be powered only by its electric motor, so it could be considered a kind of electric car.
# TODO
# 13. Full hybrids feature electric motors of about 100HP.
# TODO
# 14. The Toyota Prius is a full and parallel hybrid.
add_statement(hybrids_KB, make_atom("isa", make_const("toyota-prius"), make_const("parallel-hybrid")))
# 14. The Toyota Prius is a full and parallel hybrid.
add_statement(hybrids_KB, make_atom("isa", make_const("toyota-prius"), make_const("full-hybrid")))
# 15. The Honda Insight is a mild parallel hybrid.
# TODO
# 15. The Honda Insight is a mild parallel hybrid.
# TODO
# 16. The Chevrolet Volt is a full, series hybrid.
# TODO
# 16. The Chevrolet Volt is a full, series hybrid.
# TODO
# 17. The Nissan Leaf is an electric car.
# TODO

print("This is how the knowledge base is presented:")
print_KB(hybrids_KB)
print("==================== \n Inside the KB we can notice:")
print("\n".join([str(s) for s in hybrids_KB]))

OK: Added statement gas-power(car, 110)
OK: Added statement ako(hybrid-car, car)
OK: Added statement ako(parallel-hybrid, hybrid-car)
OK: Added statement ako(series-hybrid, hybrid-car)
OK: Added statement ako(mild-hybrid, hybrid-car)
OK: Added statement ako(full-hybrid, hybrid-car)
OK: Added statement electric-power(hybrid-car, 30)
OK: Added statement gas-power(hybrid-car, 100)
OK: Added statement ako(assisted-electric, car)
OK: Added statement electric-power(assisted-electric, 20)
OK: Added statement ako(mild-hybrid, assisted-electric)
OK: Added statement isa(toyota-prius, parallel-hybrid)
OK: Added statement isa(toyota-prius, full-hybrid)
This is how the knowledge base is presented:
KB now:
			gas-power(car, 110)
			ako(hybrid-car, car)
			ako(parallel-hybrid, hybrid-car)
			ako(series-hybrid, hybrid-car)
			ako(mild-hybrid, hybrid-car)
			ako(full-hybrid, hybrid-car)
			electric-power(hybrid-car, 30)
			gas-power(hybrid-car, 100)
			ako(assisted-electric, car)
			electric-power(assi

### Tâche 2

Mettez en œuvre les 3 fonctions supports ci-dessous. Leur fonctionnalité se réfère aux parents ou aux attributs déclarés **explicitement** pour un certain noeud (aucune traversée du réseau n'est requise).

In [ ]:
# The function receives the name of a concept from the semantic network (object or class) and the name of an attribute
# and returns the value of the attribute for that concept, if given _explicitly_.
def get_attribute(node_name, attribute_name, net):
    # TODO
    pass

# The function returns a list of the names of the classes to which the object with the given name belongs
def make_isa_ancestor_list(node_name, net):
    # TODO
    pass

# The function returns a list of class names that the class inherits with the given name
def make_ako_ancestor_list(node_name, net):
    # TODO
     pass

# Test!
testBatch['sem-net-1'] = [(testFKB(testA, get_attribute, hybrids_KB), [
    # 0
    (('car', 'gas-power'), '110'),
    (('hybrid-car', 'electric-power'), '30'),
    (('hybrid-car', 'gas-power'), '100'),
    (('electric-car', 'electric-power'), '120'),
    (('electric-car', 'gas-power'), '0'),
        ]),
                      (testFKB(testL, make_isa_ancestor_list, hybrids_KB, True), [
    # 5
    ('toyota-prius', ['full-hybrid', 'parallel-hybrid']),
    ('chevrolet-volt', ['full-hybrid', 'series-hybrid']),
    ('nissan-leaf', ['electric-car']),
            ]),
                      (testFKB(testL, make_ako_ancestor_list, hybrids_KB, True), [
    # 8
    ('hybrid-car', ['car']),
    ('mild-hybrid', ['hybrid-car', 'assisted-electric']),
    ('full-hybrid', ['hybrid-car', 'electric-car']),
        ])]

test_batch('sem-net-1')

### Tâche 3

Mettez en oeuvre la fonction `infer_attr` qui détermine la valeur héritée par un noeud pour un attribut. Récupérez l'une des valeurs qui peuvent être héritées.

BONUS: cherchez la valeur la plus proche qui peut être héritée.

In [3]:
# infer_attr finds the value of the given name attribute, inferred for the given name node. It is enough to find a value
# in any ancestor of the node (considering the condition that there is no other definition on the path between 
# the node and the ancestors of the attribute).
# It will return a tuple consisting of the value and the name of the node where the value was found
# Returns None if no value can be found
# BONUS: implement a function which returns the closest value to the given node in the semantic network.
# If two different values are found, at a distance equal to the node, the message "CONTRADICTION" is returned instead.

def infer_attr(node_name, attribute_name, net):
    # TODO
    pass
    
# Test!
testBatch['sem-net-2'] = [(testFKB(testA, infer_attr, hybrids_KB), [
    #0
    (('car', 'gas-power'), ('110', 'car')),
    (('car', 'electric-power'), None),
    (('series-hybrid', 'gas-power'), ('100', 'hybrid-car')),
    #3
    (('nissan-leaf', 'electric-power'), ('120', 'electric-car')),
    (('nissan-leaf', 'gas-power'), ('0', 'electric-car')),
    (('full-hybrid', 'electric-power'), ('100', 'full-hybrid')),
    # BONUS:
    #6
    (('chevrolet-volt', 'electric-power'), ('100', 'full-hybrid')),
    (('mild-hybrid', 'gas-power'), ('100', 'hybrid-car')),
    (('mild-hybrid', 'electric-power'), 'CONTRADICTION'),
    (('honda-insight', 'electric-power'), 'CONTRADICTION'),
    (('honda-insight', 'gas-power'), ('100', 'hybrid-car')),
            ])]
test_batch('sem-net-2')

>>> Test batch [sem-net-2]
Test 0: Failed, got < None >, should be < ('110', 'car') >.
Test 1: OK
Test 2: Failed, got < None >, should be < ('100', 'hybrid-car') >.
Test 3: Failed, got < None >, should be < ('120', 'electric-car') >.
Test 4: Failed, got < None >, should be < ('0', 'electric-car') >.
Test 5: Failed, got < None >, should be < ('100', 'full-hybrid') >.
Test 6: Failed, got < None >, should be < ('100', 'full-hybrid') >.
Test 7: Failed, got < None >, should be < ('100', 'hybrid-car') >.
Test 8: Failed, got < None >, should be < CONTRADICTION >.
Test 9: Failed, got < None >, should be < CONTRADICTION >.
Test 10: Failed, got < None >, should be < ('100', 'hybrid-car') >.
>>>  1 / 11 tests successful.
